In [4]:
def make_annotations(pos, text, labels, font_size=10, font_color='rgb(250,250,250)'):
    L=len(pos)
    if len(text)!=L:
        raise ValueError('The lists pos and text must have the same len')
    annotations = []
    for k in range(L):
        annotations.append(
            dict(
                text=labels[k], # or replace labels with a different list for the text within the circle
                x=pos[k][0], y=2*M-position[k][1],
                xref='x1', yref='y1',
                font=dict(color=font_color, size=font_size),
                showarrow=False)
        )
    return annotations

In [16]:
import plotly.graph_objects as go

import igraph
from igraph import Graph, EdgeSeq
nr_vertices = 25
v_label = list(map(str, range(nr_vertices)))
G = Graph.Tree(nr_vertices, 2) # 2 stands for children number
lay = G.layout('rt')

position = {k: lay[k] for k in range(nr_vertices)}
Y = [lay[k][1] for k in range(nr_vertices)]
M = max(Y)

es = EdgeSeq(G) # sequence of edges
E = [e.tuple for e in G.es] # list of edges

L = len(position)
Xn = [position[k][0] for k in range(L)]
Yn = [2*M-position[k][1] for k in range(L)]
Xe = []
Ye = []
for edge in E:
    Xe+=[position[edge[0]][0],position[edge[1]][0], None]
    Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]

labels = v_label

fig = go.Figure()
fig.add_trace(go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   ))
fig.add_trace(go.Scatter(x=Xn,
                  y=Yn,
                  mode='markers',
                  name='bla',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.8
                  ))
axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            )

fig.update_layout(title= 'Tree with Reingold-Tilford Layout',
              annotations=make_annotations(position, v_label, v_label),
              font_size=12,
              showlegend=False,
              xaxis=axis,
              yaxis=axis,
              margin=dict(l=40, r=40, b=85, t=100),
              hovermode='closest',
              plot_bgcolor='rgb(248,248,248)'
              )
fig.show()

In [19]:
from typing import Dict 
from collections import defaultdict

def read_bits_from_go_test(filename: str) -> Dict[str, str]:
    bits = {}
    with open(filename, 'r') as f:
        for line in f:
            key, path = line.strip().split(',')
            bits[key.rstrip("\x00")] = path
    bits = {k: v for k, v in sorted(bits.items(), key=lambda item: item[1])}
    bits = {k: v for k, v in sorted(bits.items(), key=lambda item: len(item[1]))}
    return bits

def build_edges(k, v, edge_map=defaultdict[str, str]):
    if k 
    build_edges[]

    pass

bits = read_bits_from_go_test("bits.csv")

print(bits)
for k, v in bits.items():
    print(k, v)

    root

0        


{'G': '0', 'I': '01', 'F': '10', 'H': '10', 'E': '0100', 'B': '0100', 'A': '1101', 'J': '11010', 'C': '110100', 'D': '110100'}
G 0
I 01
F 10
H 10
E 0100
B 0100
A 1101
J 11010
C 110100
D 110100


In [15]:
import igraph as ig
import plotly.graph_objects as go


edges = [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (3, 4)]
nr_vertices = len(bits)
v_label = list(map(str, range(nr_vertices)))

G = ig.Graph(nr_vertices, edges)
lay = G.layout('rt')


position = {k: lay[k] for k in range(nr_vertices)}
Y = [lay[k][1] for k in range(nr_vertices)]
M = max(Y)

# es = EdgeSeq(G) # sequence of edges
# E = [e.tuple for e in G.es] # list of edges

L = len(position)
Xn = [position[k][0] for k in range(L)]
Yn = [2*M-position[k][1] for k in range(L)]
Xe = []
Ye = []
for edge in edges:
    Xe+=[position[edge[0]][0],position[edge[1]][0], None]
    Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]



fig = go.Figure()
fig.add_trace(go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   ))
fig.add_trace(go.Scatter(x=Xn,
                  y=Yn,
                  mode='markers',
                  name='bla',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=v_label,
                  hoverinfo='text',
                  opacity=0.8
                  ))

axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            )

fig.update_layout(title= 'Jellyfish Merkle Tree',
              annotations=make_annotations(position, v_label, v_label),
              font_size=12,
              showlegend=True,
              xaxis=axis,
              yaxis=axis,
              margin=dict(l=0, r=40, b=0, t=40),
              hovermode='closest',
              plot_bgcolor='rgb(248,248,248)'
              )
fig.show()